In [1]:
from src.data.data_manager import DataManager
from hydra import initialize, compose
from omegaconf import DictConfig
import pandas as pd
import numpy as np
import polars

In [2]:
initialize(version_base=None, config_path="config", job_name="test")
cfg = compose(config_name="general.yaml", return_hydra_config=True)
dm = DataManager(cfg)

In [3]:
dm.update_mean_volume()

30_MIN 2
...Calculating average volumes...
 [ 70% ]                                                                                                                       ==                                                                                                                             ===                                                                                                                            ====                                                                                                                           =====                                                                                                                          ======                                                                                                                         =======                                                                                                                        ========                                                            

 [ 100% ] =================================================================================                                    ============================================================================================                                   =============================================================================================                                  ==============================================================================================                                 ===============================================================================================                                ================================================================================================                               =================================================================================================                              ==================================================================================================             

{'BBG000BBV4M5': 78.13333333333334,
 'BBG000BX7DH0': 24.66304347826087,
 'BBG000C7P5M7': 186.88888888888889,
 'BBG000DBD6F6': 46.48295454545455,
 'BBG000F6YPH8': 1351.248427672956,
 'BBG000FWGSZ5': 184.97076023391813,
 'BBG000GQSRR5': 48.14462809917355,
 'BBG000K3STR7': 1454.994708994709,
 'BBG000LNHHJ9': 409.34473684210525,
 'BBG000LWNRP3': 7.939597315436242,
 'BBG000MZL0Y6': 170.4747899159664,
 'BBG000NLB2G3': 17.57674418604651,
 'BBG000NLC9Z6': 237.14336917562724,
 'BBG000PKWCQ7': 368.74,
 'BBG000Q7GG57': 17.710227272727273,
 'BBG000Q7ZZY2': 2158.5055248618783,
 'BBG000QF1Q17': 4546.488126649077,
 'BBG000QFH687': 448.6891495601173,
 'BBG000QJW156': 3274.148337595908,
 'BBG000R04X57': 8609.353535353535,
 'BBG000R607Y3': 1721.3984375,
 'BBG000RG4ZQ4': 454.20723684210526,
 'BBG000RJL816': 127.125,
 'BBG000RJWGC4': 83.95714285714286,
 'BBG000RK52V1': 1578.4237288135594,
 'BBG000RMWQD4': 4039.6220472440946,
 'BBG000RP8V70': 8.575757575757576,
 'BBG000RTHVK7': 96.83150183150182,
 'BBG000S

In [3]:
data_point = []

In [9]:
for i in range(100000):
    data_point.append(list(np.random.randint(0, 10, size=9)))
print(len(data_point), data_point[0])

100000 [2, 7, 1, 2, 5, 4, 3, 0, 5]


In [10]:
data_point = polars.DataFrame(data_point, schema=[f"col_{i}" for i in range(9)])

In [11]:
dm.write_share('BBG0100R9963', data_point)

In [16]:
sh = dm.load_share('BBG0100R9963')

In [17]:
sh

buyers_count,buyers_quantity,sellers_count,sellers_quantity,price,bids,asks,weighted_bid,weighted_ask
str,str,str,str,str,str,str,str,str


In [15]:
dm.clear_share('BBG0100R9963')